In [16]:
import numpy as np
import pandas as pd
import copy

import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import MinMaxScaler

from data import fetch_dataset
from util import move_sliding_window, num_params

from model import LSTMModel
from algorithm import fedavg,fedkd, fedgkd, cadis

pd.set_option('display.max_columns', None)
pd.set_option('display.max_columns', None)
np.set_printoptions(suppress=True, floatmode='fixed')

In [17]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('running on gpu')
else:
    device = torch.device("cpu")

# Parameters

In [18]:
window_size = 90 # Define window_size period and split inputs/labels\
batch_size = 1024
label_col_index = 0

# seq_len = 90  # (timestamps)
hidden_dim = 50
n_layers = 2
lr = 0.0001
output_dim = 1

#fed train params
num_local_epochs = 1
max_rounds = 100 #nb of total rounds for training

kd_weight = 1
buffer_size = 5



# Data Preperation

In [6]:
dataframes = fetch_dataset("./heterog")

Floor0
Floor1
Floor2
Floor3
Floor4
Floor5
Floor6
Floor7
Floor8
Floor9


In [7]:
n_clients = len(dataframes) #total number of clients to partition data into

In [8]:
for _, df in dataframes.items():
    print(df.columns)
    print(df.shape)

Index(['total_demand', 'AC', 'Light', 'Plug', 'Year', 'Month', 'Day', 'Hour',
       'Minute'],
      dtype='object')
(68183, 9)
Index(['total_demand', 'AC', 'Light', 'Plug', 'Year', 'Month', 'Day', 'Hour',
       'Minute'],
      dtype='object')
(58618, 9)
Index(['total_demand', 'AC', 'Light', 'Plug', 'Year', 'Month', 'Day', 'Hour',
       'Minute'],
      dtype='object')
(61943, 9)
Index(['total_demand', 'AC', 'Light', 'Plug', 'Year', 'Month', 'Day', 'Hour',
       'Minute'],
      dtype='object')
(50086, 9)
Index(['total_demand', 'AC', 'Light', 'Plug', 'Year', 'Month', 'Day', 'Hour',
       'Minute'],
      dtype='object')
(52978, 9)
Index(['total_demand', 'AC', 'Light', 'Plug', 'Year', 'Month', 'Day', 'Hour',
       'Minute'],
      dtype='object')
(47634, 9)
Index(['total_demand', 'AC', 'Light', 'Plug', 'Year', 'Month', 'Day', 'Hour',
       'Minute'],
      dtype='object')
(60309, 9)
Index(['total_demand', 'AC', 'Light', 'Plug', 'Year', 'Month', 'Day', 'Hour',
       'Minute'],
 

# Build the training set

In [9]:
train_loader = []
test_loader = []
label_scalers = []
for _, df in dataframes.items():
    inputs_cols_indices = range(0, df.shape[1])  # use (total_demand,Year,Month,Day,Hour,Minute) columns as features
    #move the window
    inputs, labels = move_sliding_window(
        df.values,
        window_size,
        inputs_cols_indices=inputs_cols_indices,
        label_col_index=label_col_index
    )

    # Normalize the input data columns
    sc = MinMaxScaler()
    # Obtaining the scaler for the labels(usage data) so that output can be re-scaled to actual value during evaluation
    label_sc = MinMaxScaler()

    # Split data into train/test portions and combining all data into a single array
    test_portion = int(0.2 * len(inputs))

    train_x = sc.fit_transform(inputs[:-test_portion].reshape(-1, window_size * df.shape[1]))
    train_x = train_x.reshape(-1, window_size, df.shape[1])
    train_y = label_sc.fit_transform(labels[:-test_portion])

    test_x = sc.transform(inputs[-test_portion:].reshape(-1, window_size * df.shape[1]))
    test_x = test_x.reshape(-1, window_size, df.shape[1])
    test_y = label_sc.transform(labels[-test_portion:])

    # test_x.append(testx)
    # test_y.append(testy)
    label_scalers.append(label_sc)

    # pytorch data loaders
    train_data = TensorDataset(torch.from_numpy(train_x).to('cpu'), torch.from_numpy(train_y).to('cpu'))
    train_loader.append(DataLoader(train_data, batch_size=batch_size, drop_last=True))# Drop the last incomplete batch
    test_data = TensorDataset(torch.from_numpy(test_x).to('cpu'), torch.from_numpy(test_y).to('cpu'))
    test_loader.append(DataLoader(test_data, batch_size=batch_size))# Drop the last incomplete batch

    # release some memory
    del train_x, train_y
input_dim = next(iter(train_loader[0]))[0].shape[2]  # 22

(68093, 90, 9) (68093, 1)
(58528, 90, 9) (58528, 1)
(61853, 90, 9) (61853, 1)
(49996, 90, 9) (49996, 1)
(52888, 90, 9) (52888, 1)
(47544, 90, 9) (47544, 1)
(60219, 90, 9) (60219, 1)
(67334, 90, 9) (67334, 1)
(58120, 90, 9) (58120, 1)
(62212, 90, 9) (62212, 1)


# LSTM model

In [10]:
lstm = LSTMModel(input_dim, hidden_dim, output_dim, n_layers)
model_type = 'LSTM'
print(lstm)
print(num_params(lstm))

LSTMModel(
  (lstm): LSTM(9, 50, num_layers=2, batch_first=True)
  (fc): Linear(in_features=50, out_features=1, bias=True)
  (tanh): Tanh()
)
32651


## 10

In [11]:
%%time
lstm_K5_kd = copy.deepcopy(lstm)
outputs_kd1, targets_kd1, loss_kd1, smape_kd1, mae_kd1, rmse_kd1,global_model, aggregated_logits = fedavg(
    global_model = lstm_K5_kd,
    client_train_loader = train_loader,
    test_loader = test_loader,
    label_sc = label_scalers,
    n_clients = n_clients,
    batch_size = batch_size,
    num_local_epochs = num_local_epochs,
    lr = lr,
    max_rounds = 1,
    model_type = model_type,
    device = device,
)


starting avg round 0
clients:  [0 1 2 3 4 5 6 7 8 9]
round 0, starting client 1/10, id: 0
Epoch [1/1], Train Loss: 0.08848579147092575
round 0, starting client 2/10, id: 1
Epoch [1/1], Train Loss: 0.07827427296174898
round 0, starting client 3/10, id: 2
Epoch [1/1], Train Loss: 0.04822544027895978
round 0, starting client 4/10, id: 3
Epoch [1/1], Train Loss: 0.04887921210282887
round 0, starting client 5/10, id: 4
Epoch [1/1], Train Loss: 0.07544878465918507
round 0, starting client 6/10, id: 5
Epoch [1/1], Train Loss: 0.08722994411112489
round 0, starting client 7/10, id: 6
Epoch [1/1], Train Loss: 0.06851211507269675
round 0, starting client 8/10, id: 7
Epoch [1/1], Train Loss: 0.06518652053693165
round 0, starting client 9/10, id: 8
Epoch [1/1], Train Loss: 0.07374147607220548
round 0, starting client 10/10, id: 9
Epoch [1/1], Train Loss: 0.07390090316766873
calc smape: 52.677583735408966%
MAE: 11.480547639248444
RMSE: 21.74877491235681
Average Loss:  0.05401921451424303
CPU times:

In [12]:
%load_ext memory_profiler

## Fedwkd

In [14]:
# Measure memory consumption
%memit -r 1 fedkd(global_model = global_model,aggregated_logits=aggregated_logits,client_train_loader = train_loader,test_loader = test_loader,label_sc = label_scalers,n_clients = n_clients,batch_size = batch_size,num_local_epochs = num_local_epochs,lr = lr,max_rounds = 2,model_type = model_type,device = device)


starting round 1
clients:  [0 1 2 3 4 5 6 7 8 9]
round 1, starting client 1/10, id: 0


C:\Users\Admin\anaconda3\envs\fed\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1024, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/1], Train Loss: 0.05076660194768095
round 1, starting client 2/10, id: 1
Epoch [1/1], Train Loss: 0.042104214284982955
round 1, starting client 3/10, id: 2
Epoch [1/1], Train Loss: 0.02757573638033742
round 1, starting client 4/10, id: 3
Epoch [1/1], Train Loss: 0.029804467104184322
round 1, starting client 5/10, id: 4
Epoch [1/1], Train Loss: 0.041686405541329866
round 1, starting client 6/10, id: 5
Epoch [1/1], Train Loss: 0.046375913746856345
round 1, starting client 7/10, id: 6
Epoch [1/1], Train Loss: 0.038671707457050356
round 1, starting client 8/10, id: 7
Epoch [1/1], Train Loss: 0.03790610799422631
round 1, starting client 9/10, id: 8
Epoch [1/1], Train Loss: 0.03906973163700766
round 1, starting client 10/10, id: 9
Epoch [1/1], Train Loss: 0.040465107536874725


C:\Users\Admin\anaconda3\envs\fed\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


1 [0 0 0 0 0 0 0 0 0 0]
calc smape: 58.102842248629436%
MAE: 13.316796416257846
RMSE: 17.60489989085443
Average Loss:  0.036277582307633796
peak memory: 4804.77 MiB, increment: 344.13 MiB


## fedgkd

In [19]:
%%time
lstm_K5_gkd = copy.deepcopy(lstm)

CPU times: total: 0 ns
Wall time: 1 ms


In [21]:
%memit -r 1 fedgkd(global_model = lstm_K5_gkd,client_train_loader = train_loader,test_loader = test_loader,label_sc = label_scalers,n_clients = n_clients,batch_size = batch_size,num_local_epochs = num_local_epochs,lr = lr,max_rounds = 1,model_type = model_type,device = device,buffer_size=buffer_size,kd_weight=kd_weight)


Starting round 0
clients:  [0 1 2 3 4 5 6 7 8 9]
round 0, starting client 1/10, id: 0
Epoch [1/1], Train Loss: 0.014653398435703428
round 0, starting client 2/10, id: 1
Epoch [1/1], Train Loss: 0.012889184316413272
round 0, starting client 3/10, id: 2
Epoch [1/1], Train Loss: 0.01369987951572208
round 0, starting client 4/10, id: 3
Epoch [1/1], Train Loss: 0.017477316363977317
round 0, starting client 5/10, id: 4
Epoch [1/1], Train Loss: 0.012879181233030267
round 0, starting client 6/10, id: 5
Epoch [1/1], Train Loss: 0.013338164086579469
round 0, starting client 7/10, id: 6
Epoch [1/1], Train Loss: 0.014372283275774184
round 0, starting client 8/10, id: 7
Epoch [1/1], Train Loss: 0.015437479507034786
round 0, starting client 9/10, id: 8
Epoch [1/1], Train Loss: 0.01161098595087727
round 0, starting client 10/10, id: 9
Epoch [1/1], Train Loss: 0.013291690488889191
calc smape: 42.84222619820313%
MAE: 6.891559890380983
RMSE: 11.93367708297831
Average Loss:  0.01720941612216088
peak mem

## cadis

In [22]:
lstm_K5_cadis = copy.deepcopy(lstm)

In [23]:
%memit -r 1 cadis(global_model = lstm_K5_cadis,client_train_loader = train_loader,test_loader = test_loader,lambda_kd = 1,label_sc = label_scalers,n_clients = n_clients,batch_size = batch_size,num_local_epochs = num_local_epochs,lr = lr,max_rounds = max_rounds,model_type = model_type,device = device)


starting round 0
clients:  [0 1 2 3 4 5 6 7 8 9]
round 0, starting client 1/10, id: 0
Epoch [1/1], Loss: 4.6897
round 0, starting client 2/10, id: 1
Epoch [1/1], Loss: 3.5223
round 0, starting client 3/10, id: 2
Epoch [1/1], Loss: 2.3148
round 0, starting client 4/10, id: 3
Epoch [1/1], Loss: 1.9063
round 0, starting client 5/10, id: 4
Epoch [1/1], Loss: 3.0934
round 0, starting client 6/10, id: 5
Epoch [1/1], Loss: 3.2275
round 0, starting client 7/10, id: 6
Epoch [1/1], Loss: 3.2201
round 0, starting client 8/10, id: 7
Epoch [1/1], Loss: 3.3897
round 0, starting client 9/10, id: 8
Epoch [1/1], Loss: 3.3184
round 0, starting client 10/10, id: 9
Epoch [1/1], Loss: 3.5472


C:\Users\Admin\Desktop\fedwkd\util.py:152: ClusterWarning: scipy.cluster: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix
  linkage_matrix = linkage(q_matrix, method='average')


calc smape: 52.76065109079408%
MAE: 11.473552011533435
RMSE: 21.65611635112987
Average Loss:  0.05356896809089421
peak memory: 5103.39 MiB, increment: 613.57 MiB
